In [1]:
import os

In [2]:
%pwd

'c:\\Users\\dhira\\Desktop\\MLflow\\MLflow-DVC\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\dhira\\Desktop\\MLflow\\MLflow-DVC'

In [2]:
import dagshub
dagshub.init(repo_owner='DhirajRouniyar', repo_name='MLflow-DVC', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

Initialized MLflow to track repo "DhirajRouniyar/MLflow-DVC"

Repository DhirajRouniyar/MLflow-DVC initialized!

In [7]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"  # allow duplicate OpenMP runtime

import torch
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))

CUDA available: True
GPU: NVIDIA GeForce RTX 4070 Laptop GPU


In [8]:
import torch
import torch.nn as nn
from torchvision import models

# Recreate the same model architecture
model = models.vgg16(weights=None)
model.classifier = nn.Sequential(
    nn.Flatten(),
    nn.Linear(512*7*7, 2)  # must match your original model
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Load weights
model.load_state_dict(torch.load("artifacts/training/model.pt", map_location=device))
model.eval()  # for inference

C:\Users\dhira\AppData\Local\Temp\ipykernel_20300\3757099262.py:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("artifacts/training/model.

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [9]:
#Entity

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class EvaluationConfig:
    path_to_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int
    params_classes: int

In [10]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [11]:
# 5. Update the configuration manager in src config

class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_to_model="artifacts/training/model.pt",
            training_data="artifacts/data_ingestion/kidney-ct-scan-image",
            mlflow_uri="DVC.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE,
            params_classes = self.params.CLASSES
        )
        return eval_config

In [12]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms, models
import mlflow
import mlflow.pytorch
from urllib.parse import urlparse
from pathlib import Path
import json


class Evaluation:
    def __init__(self, config: EvaluationConfig, device=None):
        self.config = config
        self.device = device or ("cuda" if torch.cuda.is_available() else "cpu")
        self.model = None
        self.valid_loader = None
        self.score = None
        self.avg_loss = None
        self.accuracy = None

    def _valid_loader(self):
        """Create validation DataLoader with split=0.3"""
        transform = transforms.Compose([
            transforms.Resize(self.config.params_image_size[:2]),
            transforms.ToTensor(),
        ])

        full_dataset = datasets.ImageFolder(
            root=self.config.training_data,
            transform=transform
        )

        valid_size = int(0.3 * len(full_dataset))
        train_size = len(full_dataset) - valid_size
        _, valid_dataset = random_split(full_dataset, [train_size, valid_size])

        self.valid_loader = DataLoader(
            valid_dataset,
            batch_size=self.config.params_batch_size,
            shuffle=False
        )

    @staticmethod
    def load_model(path: Path, num_classes: int, device="cpu"):
        """Recreate architecture and load trained weights"""
        model = models.vgg16(weights=None)
        in_features = 512 * 7 * 7
        model.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features, num_classes)
        )
        model.load_state_dict(torch.load(path, map_location=device))
        model.to(device)
        model.eval()
        return model

    def evaluation(self):
        """Run evaluation and save score"""
        self.model = self.load_model(
            path=self.config.path_to_model,
            num_classes=self.config.params_classes,
            device=self.device
        )
        self._valid_loader()

        criterion = nn.CrossEntropyLoss()
        total_loss, correct, total = 0.0, 0, 0

        with torch.no_grad():
            for images, labels in self.valid_loader:
                images, labels = images.to(self.device), labels.to(self.device)
                outputs = self.model(images)
                loss = criterion(outputs, labels)

                total_loss += loss.item() * images.size(0)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        self.avg_loss = total_loss / total
        self.accuracy = correct / total
        self.score = (self.avg_loss, self.accuracy)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        with open("scores.json", "w") as f:
            json.dump(scores, f, indent=4)

    def log_into_mlflow(self):
       # Connect to DagsHub MLflow
        dagshub.init(repo_owner="DhirajRouniyar", repo_name="MLflow-DVC", mlflow=True)

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics({
                "loss": self.avg_loss,
                "accuracy": self.accuracy
            })

            # Log model to DagsHub
            mlflow.pytorch.log_model(self.model, "model", registered_model_name="TorchModel")


In [13]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2025-09-10 00:00:14,705: INFO: common: yaml_file: config\config.yaml loaded successfully]
[2025-09-10 00:00:14,706: INFO: common: yaml_file: params.yaml loaded successfully]
[2025-09-10 00:00:14,706: INFO: common: created directory at: artifacts]


C:\Users\dhira\AppData\Local\Temp\ipykernel_20300\1977927998.py:53: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(path, map_location=device)

[2025-09-10 00:00:17,876: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/repos/DhirajRouniyar/MLflow-DVC "HTTP/1.1 200 OK"]


Initialized MLflow to track repo "DhirajRouniyar/MLflow-DVC"

[2025-09-10 00:00:17,889: INFO: helpers: Initialized MLflow to track repo "DhirajRouniyar/MLflow-DVC"]


Repository DhirajRouniyar/MLflow-DVC initialized!

[2025-09-10 00:00:17,889: INFO: helpers: Repository DhirajRouniyar/MLflow-DVC initialized!]


c:\Users\dhira\miniconda3\envs\MLflow\Lib\site-packages\_distutils_hack\__init__.py:30: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
Registered model 'TorchModel' already exists. Creating a new version of this model...
2025/09/10 00:01:00 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: TorchModel, version 2
Created version '2' of model 'TorchModel'.
